In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime, timedelta

# Indicator Creation

In this notebook we will create new interesting features which we believe to be relevant for clustering purposes. The indicators we will try to create in this notebook are:
1. Account age in days (From account creation till the release of this dataset) (datetime(2022,9,29,11,0,0))
2. Account average tweets per day
3. Account highest daily tweet count
4. Account average twitter text length
5. Account tweet hashtag average
6. Account discussion creation score (Tweet to retweet ratio)
7. Account average number of mentions per tweet

In [ ]:
users_df = pd.read_csv("dataset/users.csv")
merged_df = pd.read_csv("dataset/merged_dataset.csv")

## 1. Account age in days
Creating the account age in days feature and adding it to the dataframe.

In [ ]:
merged_df["account_created"] = pd.to_datetime(merged_df["account_created"], errors="coerce") # Even though this is already done in the previous notebook, this information is turned to string when converting the merged_df to csv
merged_df["tweet_created"] = pd.to_datetime(merged_df["tweet_created"], errors="coerce")

release_of_dataset = datetime(2022,9,29,11,0,0)
merged_df['account_age_in_days'] = (release_of_dataset - merged_df['account_created']).dt.days

merged_df.head()

Making the thing on the user_df aswell
In my opinion (Paul) it's better to make it on the users df too, cause it's faster to access if we just have to make plots on this attribute

In [ ]:
users_df["created_at"] = pd.to_datetime(users_df["created_at"], errors="coerce") # Even though this is already done in the previous notebook, this information is turned to string when converting the merged_df to csv
release_of_dataset = datetime(2022,9,29,11,0,0)
users_df["account_age_in_days"] = (release_of_dataset - users_df['created_at']).dt.days

users_df.head()

# 2. Account average tweets per day

Creating the account average tweets per day and adding it to the dataframe.

In [ ]:
list_of_average_tweets_per_day = []

for elem in merged_df["statuses_count"]:
    try:
        tweets_made = elem
        age_in_days = merged_df["account_age_in_days"][elem]
        average_tweets_per_day = int(tweets_made)/int(age_in_days)
        list_of_average_tweets_per_day.append(average_tweets_per_day)
    except KeyError:
        list_of_average_tweets_per_day.append(np.nan)

merged_df["account_average_tweets_per_day"] = list_of_average_tweets_per_day
merged_df.head()

Idea, isn't better to add this attribute to the users_df ? Because it's a user caractherization, and having it in the merged_df is gonna be redundant

In [ ]:
list_of_average_tweets_per_day = []

for elem in users_df["statuses_count"]:
    try:
        tweets_made = elem
        age_in_days = users_df["account_age_in_days"][elem]
        average_tweets_per_day = int(tweets_made)/int(age_in_days)
        list_of_average_tweets_per_day.append(average_tweets_per_day)
    except KeyError:
        list_of_average_tweets_per_day.append(np.nan)

users_df["account_average_tweets_per_day"] = list_of_average_tweets_per_day
users_df.head()

Adding indicator for average number of tweets per day in which the account ACTUALLY tweeted - Gianluca 

In [ ]:
tmp = merged_df.groupby(["user_id", "tweet_age_in_days"], as_index=False).count()

# computing count
tmp = pd.DataFrame(tmp[["user_id", "tweet_age_in_days", "name"]])
count_df = tmp.rename(columns={"name": "count"})

# computing average
tmp = count_df.groupby(["user_id"], as_index=False).mean()
tmp=pd.DataFrame(tmp[["user_id", "count"]])
avg_df = tmp.rename(columns={"count": "avg", "user_id": "id"})

In [ ]:
# adding the computed avg to users dataframe
#users_df = users_df.merge(avg_df, on="id")
users_df.head()

## 3. Account highest daily tweet count

Creating the account highest daily tweet count and adding it too the dataframe.

In [ ]:
#users_df["account_created"] = pd.to_datetime(users_df["created_at"], errors="coerce") # Even though this is already done in the previous notebook, this information is turned to string when converting the merged_df to csv
#users_df.info()

# Creating the tweet age in days (for each) till the release of dataset
release_of_dataset = datetime(2022,9,29,11,0,0)

merged_df['tweet_created_just_days'] = merged_df['tweet_created'].dt.date
merged_df["tweet_created_just_days"] = pd.to_datetime(merged_df["tweet_created_just_days"], errors="coerce")
merged_df['tweet_age_in_days'] = (release_of_dataset - merged_df['tweet_created_just_days']).dt.days

merged_df.drop(columns="tweet_created_just_days", inplace=True)
merged_df.head()

In [ ]:
# Trying to see how the group by num of days works
merged_df.groupby(["user_id", "tweet_age_in_days"], as_index=False).max()

In [ ]:
# Need to finish this indicator another time - Tengel
""" 
account_high_score = 0

# Iterate through every user in the users_df and count all duplicated datetimes
for elem in users_df["id"]:
    single_account_tweets_df = pd.DataFrame([merged_df["user_id"] == elem, merged_df[]])
    merged_df.groupby("tweet_created")
"""